# DAY 3 : Agents

## single Step  Tool function calling

### Supported models (https://docs.oracle.com/en-us/iaas/Content/generative-ai/chat-models.htm) 
- cohere.command-r-08-2024
- cohere.command-r-16k
- cohere.command-r-plus
- cohere.command-r-plus-08-2024


Questions use #generative-ai-users  or #igiu-innovation-lab slack channel


In [1]:
# set up the  variables

from oci.generative_ai_inference import GenerativeAiInferenceClient
from oci.generative_ai_inference.models import OnDemandServingMode, EmbedTextDetails,CohereChatRequest, ChatDetails
import oci
import json

CONFIG_PROFILE = "AISANDBOX"
compartmentId= "ocid1.compartment.oc1..aaaaaaaaxj6fuodcmai6n6z5yyqif6a36ewfmmovn42red37ml3wxlehjmga" 

LLM_MODEL = "cohere.command-r-16k" 
PREAMBLE = """
        Analyze the problem and pick the right set of tools to answer the question
"""
MESSAGE = """
       "I'd like 4 apples and a fish please"
"""
llm_service_endpoint= "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

## Sep up the tools

In [2]:
item_param = oci.generative_ai_inference.models.CohereParameterDefinition()
item_param.description = "the item requested to be purchased, in all caps eg. Bananas should be BANANAS"
item_param.type = "str"
item_param.is_required = True

quantity_param = oci.generative_ai_inference.models.CohereParameterDefinition()
quantity_param.description = "how many of the items should be purchased"
quantity_param.type = "int"
quantity_param.is_required = True

shop_tool = oci.generative_ai_inference.models.CohereTool()
shop_tool.name = "personal_shopper"
shop_tool.description = "Returns items and requested volumes to purchase"
shop_tool.parameter_definitions = {
    "item": item_param,
    "quantity": quantity_param
}

## specify the tools to use in the chat request 

In [3]:
# oci key enabled for api access
config = oci.config.from_file('~/.oci/config', CONFIG_PROFILE)

# chat request      
llm_chat_request = CohereChatRequest()
llm_chat_request.preamble_override = PREAMBLE 
llm_chat_request.message = MESSAGE
llm_chat_request.is_stream = False 
llm_chat_request.max_tokens = 500 # max token to generate, can lead to incomplete responses
llm_chat_request.is_force_single_step = True
llm_chat_request.tools = [ shop_tool]



# set up chat details
chat_detail = ChatDetails()
chat_detail.serving_mode = OnDemandServingMode(model_id=LLM_MODEL)
chat_detail.compartment_id = compartmentId
chat_detail.chat_request = llm_chat_request

# set up the LLM client 
llm_client = GenerativeAiInferenceClient(
                config=config,
                service_endpoint=llm_service_endpoint,
                retry_strategy=oci.retry.NoneRetryStrategy(),
                timeout=(10,240))

## call the LLM 

In [4]:
step = 1
chat_response = llm_client.chat(chat_detail)
print(f"**************************Step {step} Result**************************")
print(f"message = {chat_response.data.chat_response.text}")
print(f"tool calls = {chat_response.data.chat_response.tool_calls}")

**************************Step 1 Result**************************
message = 
tool calls = [{
  "name": "personal_shopper",
  "parameters": {
    "item": "APPLES",
    "quantity": 4
  }
}, {
  "name": "personal_shopper",
  "parameters": {
    "item": "FISH",
    "quantity": 1
  }
}]


## Call the tool

Note: 
1. in this example we are not explicity calling the tool, we are just returning a made up response.  you will insert an explicit call to teh toolapi for real code
2. We have the call teh llm again with teh tool response for the final answer

In [5]:
llm_chat_request.tool_results = []
i = 0
for call in chat_response.data.chat_response.tool_calls:
        tool_result = oci.generative_ai_inference.models.CohereToolResult()
        tool_result.call = call
        # try to change response to out of stock etc for one or both items and see
        tool_result.outputs = [ { "response": "Completed, in stock" } ] 
        llm_chat_request.tool_results.append(tool_result)

chat_response = llm_client.chat(chat_detail)

# Print result
print("**************************Step 2 Result**************************")
print(f"message = {chat_response.data.chat_response.text}")
print(f"tool calls = {chat_response.data.chat_response.tool_calls}")

**************************Step 2 Result**************************
message = That's no problem! I've checked and we have both apples and fish in stock. Can I help you with anything else?
tool calls = None


# Streaming version 

Steaming response reduces latency, specially if response has a lot of text. but its involved as we have to process events 

we first define the function to process the events


PS: this seems to be broken. refer to python code ( command_r_tool_single_step_demo_streaming.py) for the implementation

In [6]:
def get_tool_calls(chat_response):
    for event in chat_response.data.events():
        res = json.loads(event.data)
        text = res['text']
        if 'finishReason' in res:
            if 'toolCalls' in res:
                #print(f"\ntools to use : {res['toolCalls']}",flush=True)
                return text,res['toolCalls']
            else:
                return text,None
        else:
            if 'text' in res:
                print(res['text'], end="", flush=True)
    print("\n")
    return None,None


def get_tool_calls(chat_response):
    for event in chat_response.data.events():
        res = json.loads(event.data)
        if 'finishReason' in res:
            print(f"\ntools to call: {res['toolCalls']}")
            return res['toolCalls']
        if 'text' in res:
            print(res['text'], end="", flush=True)
    print("\n")


### call the llm in streaming mode 


In [7]:
llm_chat_request.is_stream = True
step =1 
chat_response = llm_client.chat(chat_detail)

text,tool_calls= get_tool_calls(chat_response)
print(f"\n **************************Step {step} Result**************************")
print(f"message = {text}")
print(f"tool calls = {tool_calls}")



That's no problem! I've checked and we have both apples and fish in stock. I'll grab you 4 apples and 1 fish.

Can I help you with anything else?

KeyError: 'toolCalls'

# call tools

In [ ]:
llm_chat_request.tool_results = []
for call in tool_calls:
    tool_result = oci.generative_ai_inference.models.CohereToolResult()
    tool_result.call = call
    tool_result.outputs = [ { "response": "Completed" } ] 
    llm_chat_request.tool_results.append(tool_result)

step = step+1
chat_response = llm_client.chat(chat_detail)
text,tool_calls= get_tool_calls(chat_response)
print(f"message = {text}")
print(f"tool calls = {tool_calls}")


## Exercise : Weather Ansering App

1. Create an App that  gives the weather information for a given city 
    * Eg:  	A. Lucknow will have temp around 25 degrees and will rain 

2. calls the weather API for a given city
    * Api hosted at : https://ic-edge.ugbu.oraclepdemos.com/ash/docs
        * Input 
            * City Name  
            * DatDays in future
        * Output
            * Low
            * High
            * Chance of rain 
3. Types of questions supported
    * What is the weather in lucknow tomm
    * What will be highs in capital of india
    * Which major city in Karnataka will it rain tomm 
